In [1]:
import doctest
import moarchiving
# reload(moarchiving)
doctest.testmod(moarchiving)

TestResults(failed=0, attempted=17)

In [2]:
a = moarchiving.BiobjectiveNondominatedSortedList(
    [[-0.749, -1.188], [-0.557, 1.1076],
     [0.2454, 0.4724], [-1.146, -0.110]])
a

[[-1.146, -0.11], [-0.749, -1.188]]

In [3]:
a.add([-1, -3])

1

In [4]:
a

[[-1.146, -0.11], [-1, -3]]

In [5]:
a.add([-1.5, 44])

0

In [6]:
a

[[-1.5, 44], [-1.146, -0.11], [-1, -3]]

In [7]:
a.dominates(a[0])

True

In [8]:
a.dominates([-1.2, 1])

False

In [9]:
a._asserts()